In [1]:
import tensorflow as tf
import melt 
import gezi
import numpy as np

from tqdm.auto import tqdm
from tqdm import tqdm
from tqdm import tqdm as tqdm_base
def tqdm(*args, **kwargs):
    if hasattr(tqdm_base, '_instances'):
        for instance in list(tqdm_base._instances):
            tqdm_base._decr_instances(instance)
    return tqdm_base(*args, **kwargs)

tensorflow_version: 1.14.0
torch_version: 1.3.1
Using TensorFlow backend.


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
dir = '/home/gezi/tmp/rank/data/video_hour_sgsapp_v1'

In [3]:
feats = [x.strip().split('\t') for x in open(dir + '/2019111214/feature_index').readlines()]

In [4]:
model_dir = dir + '/2019111214'

In [5]:
from tensorflow.python import pywrap_tensorflow
model_path = melt.get_model_path(model_dir)
print(model_path)
reader = pywrap_tensorflow.NewCheckpointReader(model_path)

/home/gezi/tmp/rank/data/video_hour_sgsapp_v1/2019111214/model.ckpt-1.00-897


In [6]:
checkpoint = reader.get_variable_to_shape_map()

In [7]:
lookups = reader.get_tensor('hash_embedding/lookups')

In [8]:
lookups

array([[      0,       0],
       [1227554, 1205357],
       [2759398,  946202],
       ...,
       [1457130,  653612],
       [2251141,  105752],
       [2795490, 2059493]], dtype=int32)

In [9]:
deep_weight =  reader.get_tensor('wide_deep/deep/emb/embeddings')

In [10]:
deep_weight

array([[-0.03680759, -0.03796738,  0.01173502, ...,  0.03835703,
        -0.00205764,  0.01275528],
       [-0.02140581, -0.01609494,  0.04086716, ...,  0.00257335,
        -0.04092364, -0.09978314],
       [ 0.00541245, -0.07558268, -0.01758448, ..., -0.03137203,
        -0.01085704,  0.00904625],
       ...,
       [-0.0583233 , -0.04497235,  0.0062352 , ...,  0.03827355,
        -0.04021412,  0.01332126],
       [-0.03699796,  0.02887941,  0.08264269, ..., -0.00075859,
        -0.04886567, -0.00998549],
       [ 0.04471377, -0.04138492,  0.00125036, ..., -0.03761692,
         0.01434598, -0.06310306]], dtype=float32)

In [11]:
x = deep_weight.reshape(-1)

In [12]:
sum(x * x) / 2

209338.00519917975

In [13]:
fids = set()

In [14]:
for feat, count in tqdm(feats):
    count = int(count)
    if count >= 10:
      fid = gezi.hash_int64(feat) % 20000000
#       if fid == 0:
#         print(feat, fid)
      index1, index2 = lookups[fid]
      fids.add(index1)
      fids.add(index2)

100%|██████████| 16760871/16760871 [00:28<00:00, 587234.05it/s]


In [15]:
len(fids)

2529154